In [1]:
from cycling_manager.preprocess import get_data, preprocess, split
from cycling_manager.new_sequences import get_full_sequence, get_images

In [2]:
df = preprocess(get_data())

full df shape (603619, 25)
full df shape (603619, 25)
Score vertical meters - model: DecisionTreeRegressor() score: 0.9995662844000273
Score vertical meters - model: DecisionTreeRegressor() score: 0.9999208844300826
Score vertical meters - model: KNeighborsClassifier() score: 0.9935552723467305


In [3]:
train, test = split(df, 2020, 2022)
train.head()

,name,year,race_name
472,magnus-cort-nielsen,2021,tour-de-france
634,maciej-bodnar,2021,giro-d-italia
622,jan-tratnik,2021,giro-d-italia
623,rafael-valls,2021,giro-d-italia
624,giovanni-visconti,2021,giro-d-italia


In [70]:
X_decoder_num, y_decoder, X_encoder_num_no_y, y_encoder_num, X_decoder_img_ls, X_encoder_img_ls, y_encoder_img_ls=\
    get_full_sequence(df, 'magnus-cort-nielsen', 2021, "tour-de-france", 40, 30, img=True, binary=True, y_encoder=True)

In [72]:
X_decoder_num.shape, y_decoder.shape, X_encoder_num_no_y.shape, y_encoder_num.shape

(TensorShape([21, 5]),
 TensorShape([21, 1]),
 TensorShape([40, 14]),
 TensorShape([40, 1]))

In [53]:
X_encoder_img, X_decoder_img, y_encoder_img, to_drop_ls = get_images(X_decoder_img_ls, X_encoder_img_ls, y_encoder_img_ls, resnet=True)